# **Step 1: Setting Up Your Environment**

In [ ]:
! pip install pathway bokeh pandas numpy matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 22

# **Step 2: Importing Libraries & Dataset**

In [ ]:
import pandas as pd
import numpy as np
import pathway as pw
import bokeh.plotting
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv("/content/dataset.csv")
df['datetime'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

# **Step 3: Preprocessing the Data**

In [ ]:
# Preprocess the dataset
df = df.sort_values(['SystemCodeNumber', 'datetime'])

# Encode vehicle type for demand function (optional based on your needs)
vehicle_weights = {'car': 1.0, 'bike': 0.7, 'truck': 1.3, 'cycle': 0.5}
df['vehicle_weight'] = df['VehicleType'].map(vehicle_weights)

# **Step 4: Developing Model 1 — Baseline Linear Pricing**

In [ ]:
df['price_baseline'] = 10.0  # Initialize with a float value
alpha = 2  # Linear coefficient for price adjustment
prev_price = {}

for idx, row in df.iterrows():
    lot = row['SystemCodeNumber']
    occ = row['Occupancy']
    cap = row['Capacity']

    prev = prev_price.get(lot, 10.0) # Initialize with a float value
    new_price = prev + alpha * (occ / cap)

    df.at[idx, 'price_baseline'] = new_price
    prev_price[lot] = new_price

# **Step 5: Developing Model 2 — Demand-Based Pricing**

In [ ]:
# Demand function coefficients (you can adjust them based on experimentation)
a1, a2, a3, a4, a5 = 1.0, 0.5, 0.3, 1.0, 1.2
lambda_param = 0.3
base_price = 10

df['price_demand'] = 10.0 # Initialize with a float value

for idx, row in df.iterrows():
    occ_rate = row['Occupancy'] / row['Capacity']
    demand = (a1 * occ_rate + a2 * row['QueueLength'] - a3 * row['TrafficConditionEncoded'] +
              a4 * row['IsSpecialDay'] + a5 * row['vehicle_weight'])

    demand_norm = np.tanh(demand)  # Normalize demand to [-1,1]
    price = base_price * (1 + lambda_param * demand_norm)
    price = np.clip(price, 5, 20)

    df.at[idx, 'price_demand'] = price

In [ ]:
# Encode TrafficConditionNearby
traffic_mapping = {'low': 0, 'average': 1, 'high': 2} # Example mapping
df['TrafficConditionEncoded'] = df['TrafficConditionNearby'].map(traffic_mapping)

# **Step 6: Developing Model 3 — Competitive Pricing**

In [ ]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, sqrt, atan2
from sklearn.neighbors import BallTree

# Convert lat/lon to radians for haversine
df['lat_rad'] = np.radians(df['Latitude'])
df['lon_rad'] = np.radians(df['Longitude'])

# Create unique list of lots with coordinates
lots_df = df[['SystemCodeNumber', 'Latitude', 'Longitude', 'lat_rad', 'lon_rad']].drop_duplicates('SystemCodeNumber')

# Build spatial tree for fast neighbor lookup
coords = lots_df[['lat_rad', 'lon_rad']].values
tree = BallTree(coords, metric='haversine')

# Radius = 1km in radians
radius = 1 / 6371

# Query neighbors for each lot
indices = tree.query_radius(coords, r=radius)

# Build mapping: {lot_id: [neighbor_lots]}
neighbor_map = {}
for i, lot_id in enumerate(lots_df['SystemCodeNumber'].values):
    neighbor_ids = lots_df.iloc[indices[i]]['SystemCodeNumber'].values
    neighbor_ids = neighbor_ids[neighbor_ids != lot_id]  # remove self
    neighbor_map[lot_id] = neighbor_ids

# Prepare for fast lookup
df['price_competitive'] = df['price_demand']  # fallback default

# Optimize by processing each timestamp group at once
for time, time_group in df.groupby('datetime'):
    time_prices = time_group.set_index('SystemCodeNumber')['price_demand'].to_dict()
    time_occ = time_group.set_index('SystemCodeNumber')['Occupancy'].to_dict()
    time_cap = time_group.set_index('SystemCodeNumber')['Capacity'].to_dict()

    for lot_id in time_group['SystemCodeNumber']:
        my_price = time_prices.get(lot_id, 10)
        occ = time_occ.get(lot_id, 0)
        cap = time_cap.get(lot_id, 1)

        neighbors = neighbor_map.get(lot_id, [])
        neighbor_prices = [time_prices[nid] for nid in neighbors if nid in time_prices]

        if neighbor_prices:
            avg_comp_price = np.mean(neighbor_prices)
            if occ >= cap:
                final_price = min(my_price, avg_comp_price)
            elif avg_comp_price > my_price:
                final_price = min(my_price + 1.5, 20)
            else:
                final_price = my_price
        else:
            final_price = my_price

        df.loc[(df['SystemCodeNumber'] == lot_id) & (df['datetime'] == time), 'price_competitive'] = final_price


# **Step 7: Visualization Using Bokeh**

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

output_notebook()

lot_sample = df[df['SystemCodeNumber'] == 'BHMBCCMKT01'] # Using a sample SystemCodeNumber
source = ColumnDataSource(lot_sample)

p = figure(x_axis_type="datetime", title="Price Over Time for BHMBCCMKT01", height=300)
p.line('datetime', 'price_baseline', source=source, legend_label="Baseline", color="blue")
p.line('datetime', 'price_demand', source=source, legend_label="Demand-Based", color="green")
p.line('datetime', 'price_competitive', source=source, legend_label="Competitive", color="red")
p.legend.location = "top_left"
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price ($)"

show(p)

# **Step 8: Pathway Integration for Real-Time Simulation**

In [ ]:
import pathway as pw
import numpy as np

# Define the schema for the input data
class InputSchema(pw.Schema):
    SystemCodeNumber: str
    LastUpdatedDate: str
    LastUpdatedTime: str
    Latitude: float
    Longitude: float
    Occupancy: int
    Capacity: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: int
    ID: int # Assuming ID is also in the input if needed

# Define the calculate_demand function as a UDF
@pw.udf
def calculate_demand(
    system_code_number: str,
    last_updated_date: str,
    last_updated_time: str,
    latitude: float,
    longitude: float,
    occupancy: int,
    capacity: int,
    vehicle_type: str,
    traffic_condition_nearby: str,
    queue_length: int,
    is_special_day: int,
    id: int
) -> float:
    # Need to handle the categorical 'TrafficConditionNearby'
    traffic_mapping = {'low': 0, 'average': 1, 'high': 2}
    traffic_encoded = traffic_mapping.get(traffic_condition_nearby, 0)

    vehicle_weights = {'car': 1.0, 'bike': 0.7, 'truck': 1.3, 'cycle': 0.5}
    vehicle_weight = vehicle_weights.get(vehicle_type, 1.0)

    occ_rate = occupancy / capacity
    a1, a2, a3, a4, a5 = 1.0, 0.5, 0.3, 1.0, 1.2

    demand = (a1 * occ_rate + a2 * queue_length - a3 * traffic_encoded +
              a4 * is_special_day + a5 * vehicle_weight)
    return float(demand)

# Define a UDF for calculating the price from demand
@pw.udf
def calculate_price(demand: float) -> float:
    lambda_param = 0.3
    base_price = 10.0
    demand_norm = np.tanh(demand)
    price = base_price * (1.0 + lambda_param * demand_norm)
    # Apply clipping within the UDF
    return float(np.clip(price, 5.0, 20.0))


# Build the Pathway pipeline
input_stream = pw.io.csv.read("/content/dataset.csv", schema=InputSchema, autocommit_duration_ms=50)

# Add datetime column
input_stream = input_stream.with_columns(
    datetime=pw.this.LastUpdatedDate + ' ' + pw.this.LastUpdatedTime
)

# Calculate demand using the calculate_demand UDF
demand_stream = input_stream.with_columns(
    demand=calculate_demand(
        pw.this.SystemCodeNumber,
        pw.this.LastUpdatedDate,
        pw.this.LastUpdatedTime,
        pw.this.Latitude,
        pw.this.Longitude,
        pw.this.Occupancy,
        pw.this.Capacity,
        pw.this.VehicleType,
        pw.this.TrafficConditionNearby,
        pw.this.QueueLength,
        pw.this.IsSpecialDay,
        pw.this.ID
    )
)

# Calculate price based on demand using the calculate_price UDF
final_pricing_stream = demand_stream.with_columns(
    final_price = calculate_price(pw.this.demand)
)

# Select the relevant columns for output
output_stream = final_pricing_stream.select(
    lot_id = pw.this.SystemCodeNumber,
    timestamp = pw.this.datetime,
    final_price = pw.this.final_price
)

# Let's uncomment pw.run() for the user to execute the pipeline
pw.run()

Output()

# This is a Title

You can use single or multiple '#' symbols to create different levels of headings (titles and subtitles) in Markdown cells.

For example:

# Main Title
## Section Title
### Sub-section Title

Just create a new text cell and type your title using the '#' symbol followed by a space.

# Task
Generate a comprehensive README.md file for a GitHub repository based on the provided problem statement in "/content/problem statement.pdf" and the solution code in the current notebook cell. The README should include a project overview, tech stack, architecture diagram (with Mermaid code), a detailed explanation of the architecture and workflow, and suggestions for additional relevant documentation.

## Create a readme.md file

### Subtask:
Generate a comprehensive README file that includes a project overview, tech stack, architecture diagram (with instructions on how to generate it using Mermaid), and a detailed explanation of the architecture and workflow.


**Reasoning**:
I need to generate the content for the README.md file based on the instructions. This involves writing the sections for Project Overview, Tech Stack, Architecture Diagram (with instructions for Mermaid), and Architecture Explanation. I will combine these into a single string and then write it to a file named `README.md`.



In [40]:
readme_content = """# Dynamic Parking Pricing Models

## Project Overview

This project explores and implements different dynamic pricing strategies for parking lots based on real-time data such as occupancy, queue length, traffic conditions, and nearby parking lot prices. The goal is to develop models that can adjust parking prices dynamically to optimize revenue and manage parking space utilization effectively. The project implements three distinct pricing models: a baseline linear model, a demand-based model, and a competitive pricing model, culminating in a real-time simulation pipeline using the Pathway library.

## Tech Stack

*   **pandas:** Used for data manipulation and analysis.
*   **numpy:** Used for numerical operations, especially in price calculations and array manipulation.
*   **bokeh:** Used for interactive data visualization to compare the pricing models.
*   **pathway:** Used for building a real-time data processing pipeline for dynamic pricing simulation.
*   **scikit-learn:** Specifically `BallTree` is used for efficient spatial lookups in the competitive pricing model.
*   **math:** Used for mathematical operations, particularly in geographic calculations.

## Architecture

The project architecture is designed to process parking data, apply different pricing models, and simulate a real-time pricing system.

### Architecture Diagram

The following diagram illustrates the data flow and components of the dynamic pricing system. You can visualize this diagram by copying the following Mermaid code into a Mermaid-compatible viewer or markdown renderer (like on GitHub or in a live editor).

```mermaid
graph TD
    A[Input Data (CSV)] --> B(Data Preprocessing);
    B --> C{Pricing Models};
    C --> D1[Baseline Linear Pricing];
    C --> D2[Demand-Based Pricing];
    C --> D3[Competitive Pricing];
    D1 --> E[Output Prices];
    D2 --> E;
    D3 --> E;
    B --> F[Pathway Real-Time Simulation];
    F --> G(UDFs: Calculate Demand, Calculate Price);
    G --> H[Real-time Pricing Output];
    E --> I(Visualization);
    F --> H;
```

### Architecture and Workflow Explanation

The workflow begins with loading the parking data from a CSV file into a pandas DataFrame.

1.  **Data Preprocessing:** The data is sorted by `SystemCodeNumber` and `datetime` to ensure chronological processing. A `datetime` column is created by combining the date and time columns. Vehicle types are encoded with weights for the demand function, and traffic conditions are also encoded numerically.

2.  **Pricing Models:** Three distinct pricing models are implemented:
    *   **Baseline Linear Pricing:** This model adjusts the price linearly based on the occupancy rate of a parking lot. It maintains a history of the previous price for each lot and increments it based on the current occupancy relative to capacity.
    *   **Demand-Based Pricing:** This model calculates a 'demand' score based on various factors including occupancy rate, queue length, traffic conditions, special events, and vehicle type. This demand score is then normalized using the hyperbolic tangent function, and the price is adjusted based on this normalized demand and a base price. The price is clipped to a defined range.
    *   **Competitive Pricing:** This model considers the prices of nearby parking lots. It first identifies neighboring lots within a certain radius using spatial indexing (`BallTree`). Then, for each parking lot at a given timestamp, it calculates the average price of its neighbors. If a lot is fully occupied, its price is set to the minimum of its current demand-based price and the average competitive price. If the average competitive price is higher than the lot's current price, the lot's price is increased slightly (capped at a maximum). Otherwise, the demand-based price is maintained.

3.  **Visualization:** Bokeh is used to visualize the prices generated by the different models over time for a selected parking lot, allowing for a comparison of their behavior.

4.  **Pathway Real-Time Simulation:** Pathway is integrated to demonstrate how the dynamic pricing could work in a real-time streaming scenario.
    *   A Pathway pipeline is built to read the data as a stream.
    *   User-Defined Functions (UDFs) are defined for `calculate_demand` and `calculate_price`, mirroring the logic from the pandas implementation but designed for Pathway's data streaming model.
    *   The pipeline processes the input stream, calculates demand using the UDF, and then calculates the final price using the pricing UDF.
    *   The relevant output (lot ID, timestamp, and final price) is selected for a potential output sink.
    *   `pw.run()` is called to execute the real-time pipeline.

This architecture allows for the exploration of different pricing strategies and provides a foundation for building a real-time dynamic parking pricing system.
"""

with open("README.md", "w") as f:
    f.write(readme_content)

## Generate the architecture diagram code

### Subtask:
Provide the Mermaid code for the architecture diagram based on the provided notebook.


**Reasoning**:
Extract the Mermaid code block from the `readme_content` string using regular expressions.



In [41]:
import re

mermaid_match = re.search(r"```mermaid\n([\s\S]*?)\n```", readme_content)
if mermaid_match:
    mermaid_code = mermaid_match.group(1)
    print(mermaid_code)
else:
    print("Mermaid code block not found.")

graph TD
    A[Input Data (CSV)] --> B(Data Preprocessing);
    B --> C{Pricing Models};
    C --> D1[Baseline Linear Pricing];
    C --> D2[Demand-Based Pricing];
    C --> D3[Competitive Pricing];
    D1 --> E[Output Prices];
    D2 --> E;
    D3 --> E;
    B --> F[Pathway Real-Time Simulation];
    F --> G(UDFs: Calculate Demand, Calculate Price);
    G --> H[Real-time Pricing Output];
    E --> I(Visualization);
    F --> H;


## Suggest additional documentation

### Subtask:
Recommend any other relevant documentation that could be included in the repository.


## Review and finalize

### Subtask:
Advise the user to review all the generated content and organize their repository before submitting it to GitHub.


**Reasoning**:
Remind the user to review the generated content, organize the repository, add comments, test the code, and consider a .gitignore file.



In [42]:
print("Please review the generated README.md file carefully for accuracy, clarity, and completeness.")
print("Consider organizing your repository with a logical directory structure for your code files, README.md, the problem statement PDF, and any other relevant assets.")
print("Adding comments to your code can significantly improve readability and understanding for others (and your future self!).")
print("It's recommended to test the Bokeh visualization and Pathway simulation code in your target environment (e.g., GitHub Codespaces, a local setup) to ensure they run correctly.")
print("Finally, consider adding a .gitignore file to exclude any unnecessary files or directories from being committed to your repository.")

Please review the generated README.md file carefully for accuracy, clarity, and completeness.
Consider organizing your repository with a logical directory structure for your code files, README.md, the problem statement PDF, and any other relevant assets.
Adding comments to your code can significantly improve readability and understanding for others (and your future self!).
It's recommended to test the Bokeh visualization and Pathway simulation code in your target environment (e.g., GitHub Codespaces, a local setup) to ensure they run correctly.
Finally, consider adding a .gitignore file to exclude any unnecessary files or directories from being committed to your repository.


## Summary:

### Data Analysis Key Findings

*   The project successfully generated a comprehensive `README.md` file for a GitHub repository, including sections for project overview, tech stack, architecture diagram with Mermaid code, and a detailed explanation of the architecture and workflow.
*   The Mermaid code for the architecture diagram was successfully extracted from the generated `README.md` content.
*   The solving process successfully provided recommendations for additional relevant documentation for the repository and advice on reviewing and organizing the repository before submitting it to GitHub.

### Insights or Next Steps

*   Ensure the Mermaid code renders correctly in a GitHub environment by previewing the generated `README.md`.
*   Implement the suggested additional documentation (e.g., contributing guidelines, licensing information, installation instructions) to enhance the repository's completeness.
